In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LinearRegression").getOrCreate()

In [ ]:
df = spark.read.csv(path = "/kaggle/input/pyspark-ml/cruise_ship_info.csv", 
                    inferSchema = True, header = True)

df.show()

In [ ]:
from pyspark.ml.feature import StringIndexer

indexer1 = StringIndexer(inputCol = "Cruise_line",
                         outputCol = "Cruise_line_Index")

indexed1 = indexer1.fit(df).transform(df)

indexed1.show()

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ["Age", "Tonnage", "passengers", "length",
                                         "cabins", "passenger_density", "Cruise_line_Index"],
                            outputCol = "features")

output = assembler.transform(indexed1)

output.show(truncate = False)

In [ ]:
train, test = output.randomSplit(weights = [0.8, 0.2], seed = 42)

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
lr = LinearRegression(featuresCol = "features",
                      labelCol = "crew",
                      predictionCol = "prediction")

model = lr.fit(train)

In [ ]:
y_hat = model.evaluate(test)

y_hat.residuals.show()

In [ ]:
print("RMSE: {}".format(y_hat.rootMeanSquaredError))
print("MSE: {}".format(y_hat.meanSquaredError))
print("r2: {}".format(y_hat.r2))